In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
import json
from pandas.io.json import json_normalize

In [4]:
# load json as string
sample_json_s = json.load((open('data/world_bank_projects.json')))
sample_json_n = json_normalize(sample_json_s)
sample_json_n.head(2)

,_id.$oid,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,source,status,supplementprojectflg,theme1.Name,theme1.Percent,theme_namecode,themecode,totalamt,totalcommamt,url
0,52b213b38594d8a2be17c780,1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,IBRD,Active,N,Education for all,100,"[{'code': '65', 'name': 'Education for all'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,52b213b38594d8a2be17c781,2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,...,IBRD,Active,N,Other economic management,30,"[{'code': '24', 'name': 'Other economic manage...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en


In [5]:
# load as Pandas dataframe
sample_json_df = pd.read_json('data/world_bank_projects.json')
sample_json_df.head(2)

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
0,{'$oid': '52b213b38594d8a2be17c780'},1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,"ET,BS,ES,EP",IBRD,Active,N,"{'Name': 'Education for all', 'Percent': 100}","[{'code': '65', 'name': 'Education for all'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,{'$oid': '52b213b38594d8a2be17c781'},2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,...,"BZ,BS",IBRD,Active,N,"{'Name': 'Other economic management', 'Percent...","[{'code': '24', 'name': 'Other economic manage...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en


## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,

* Find the 10 countries with most projects
* Find the top 10 major project themes (using column 'mjtheme_namecode')
* In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

## Answer 1

In [6]:
# assigning dataframe sample_json_df to df
df = sample_json_df
#segregating columns countryshortname and project_name
df = df[['countryshortname', 'project_name']]
#groupby countryshortname and aggregate project_name
df = df.groupby('countryshortname').agg([pd.Series.nunique])
#sort the values with highest occurrence at the top
df.sort_values([('project_name', 'nunique')], ascending = False).head(10)

,project_name
,nunique
countryshortname,
China,19
Indonesia,19
Vietnam,17
India,16
"Yemen, Republic of",13
Nepal,12
Bangladesh,12
Morocco,12


## Answer 2

In [7]:
s = sample_json_s
n = json_normalize(sample_json_s, 'mjtheme_namecode')
g = n.groupby(['name', 'code'])['code', 'name'].count().sort_values('name', ascending = False).drop('code', 1).rename(columns = {'name':'count'}).reset_index('name')
g[g.name != ''].head(10)

,name,count
code,,
11,Environment and natural resources management,223
10,Rural development,202
8,Human development,197
2,Public sector governance,184
6,Social protection and risk management,158
4,Financial and private sector development,130
7,Social dev/gender/inclusion,119
5,Trade and integration,72
9,Urban development,47


## Answer 3

In [16]:
d = json_normalize(s, 'mjtheme_namecode')
reference = d.copy()
reference['name'] = reference['name'].replace('', np.nan)
reference = reference.dropna()
reference = reference.drop_duplicates()
reference

data = pd.merge(d, reference, on = 'code')
data = data.drop('name_x', 1)
data.columns = ['code', 'name']
data.groupby('name').size().sort_values(ascending = False).head(10)

name
Environment and natural resources management    250
Rural development                               216
Human development                               210
Public sector governance                        199
Social protection and risk management           168
Financial and private sector development        146
Social dev/gender/inclusion                     130
Trade and integration                            77
Urban development                                50
Economic management                              38
dtype: int64

In [34]:
#Alternative

df = json_normalize(sample_json_s, 'mjtheme_namecode')
df = df.sort_values('code').replace('', np.NaN)
df = df.fillna(method = 'bfill')
df.groupby('name').size().sort_values(ascending = False).head(10)

name
Environment and natural resources management    250
Rural development                               216
Human development                               210
Public sector governance                        199
Social protection and risk management           168
Financial and private sector development        146
Social dev/gender/inclusion                     130
Trade and integration                            77
Urban development                                50
Economic management                              38
dtype: int64